<a href="https://colab.research.google.com/github/Luisamfer/MBA_IA_Generativa/blob/main/Chatbot_norma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Comando para baixar o PDF

In [ ]:
#!pip install langchain faiss-cpu sentence-transformers
#!pip install langchain-community
#!pip install langchain_huggingface
#!pip install chromadb
#!pip install langchain_openai



In [ ]:
!wget -O portaria19.html http://www.in.gov.br/web/dou/-/portaria-n-19-de-21-de-marco-de-2025-619527337

In [ ]:
from langchain_community.document_loaders import BSHTMLLoader

loader = BSHTMLLoader("portaria19.html")
dados = loader.load()
#print(data[0].page_content)

## Código para popular um Banco de Dados Vetorial

In [ ]:
!pip install hf_xet

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
textos = text_splitter.split_documents(dados)
embedding_engine = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_db = Chroma.from_documents(textos, embedding_engine, persist_directory="/content/drive/MyDrive/Colab Notebooks/IA_GEN")

## Código para montar um RAG

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
n_documentos = 5

def format_docs(documentos):
    return "\n\n".join(documento.page_content for documento in documentos)

llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model = "gpt-4o-mini")
prompt = hub.pull("rlm/rag-prompt")

rag = (
    {
        "question": RunnablePassthrough(),
        "context": vector_db.as_retriever(k = n_documentos)
                    | format_docs
    }
    | prompt
    | llm
    | StrOutputParser()
)


## Código para perguntar ao RAG

In [ ]:
prompt = "Qual a documentação obrigatória para inscrição no CAF?"
rag.invoke(prompt)